# Car Prices Model building and tuning






In [1]:
# Import
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd

import tensorflow as tf
import argparse

import tempfile
import subprocess




In [ ]:
#import sys

In [ ]:
#sys.argv[1:]

In [2]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
#ot sebyachina
import os

In [4]:
path = 'C:\\Mikayel\\MK_Organizer\\A_interview_2018_Armenia\\car_data'
os.chdir(path)

In [5]:
#df = pd.read_csv('car_prices_66rows_6_cols_medi.csv')
#df.head()

In [6]:
names = [
    'price',
    'year',
    'symboling',
    'make',
    'type',
    'tuning',    
]

# We also have to specify dtypes.
dtypes = {
    'price': np.float32,
    'year': np.float32,
    'symboling': np.int32,    
    'make': str,
    'type': str,
    'tuning': str,
}

In [7]:
# Import data
df = pd.read_csv('car_prices_66rows_6_cols_medi.csv', names=names, dtype=dtypes, na_values="?")

df.head()



,price,year,symboling,make,type,tuning
0,9093.0,2008.0,2,Accord,sedan,advanced
1,8700.0,2008.0,2,Accord,sedan,basic
2,9545.0,2008.0,2,Accord,sedan,fullyloaded
3,9400.0,2008.0,2,Accord,sedan,fullyloaded
4,7250.0,2008.0,1,Civic,compact,basic


In [8]:
#df = pd.read_csv('car_data_adapted_rel_cand.csv', names=names, dtype=dtypes, na_values="?")
data = pd.read_csv('car_prices_66rows_6_cols_medi.csv', names=names, dtype=dtypes, na_values="?")



In [9]:
from sklearn import preprocessing

In [10]:
#make = tf.feature_column.categorical_column_with_hash_bucket('make', 3)
car_make = tf.feature_column.categorical_column_with_vocabulary_list('make',
                                                                          vocabulary_list=['Civic', 'Accord','CR-V'])
car_type = tf.feature_column.categorical_column_with_vocabulary_list('type',
                                                                          vocabulary_list=['sedan', 'compact','suv'])
car_tuning = tf.feature_column.categorical_column_with_vocabulary_list('tuning',
                                                                           vocabulary_list=['basic', 'advanced','fullyloaded'])

le_make = preprocessing.LabelEncoder().fit(data['make'])
data['make'] = le_make.transform(data['make'])

le_type = preprocessing.LabelEncoder().fit(data['type'])
data['type'] = le_type.transform(data['type'])

le_tuning = preprocessing.LabelEncoder().fit(data['tuning'])
data['tuning'] = le_tuning.transform(data['tuning'])


#tf.feature_column.embedding_column(make, 4)
tf.feature_column.indicator_column(car_make)
tf.feature_column.indicator_column(car_type)
tf.feature_column.indicator_column(car_tuning)
tf.feature_column.numeric_column('symboling')
tf.feature_column.numeric_column('year')


_NumericColumn(key='year', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [11]:
# Dimensions of dataset
n = data.shape[0]
p = data.shape[1]


In [12]:
# Make data a np.array - converts like
# array([[  3,  94,  31],
#       [ 29, 170, 115]], dtype=int64)
#data_price = data.loc[:, :'price'] 

data_price = data.loc[:, :'price']
data_price = data_price.values / 1000
#    data = data.values
    
# Scale price column
#scaler = MinMaxScaler(feature_range=(-1, 1))
#scaler.fit(data_price)
#data_price = scaler.transform(data_price)
#transform price vector back to df
price_df = pd.DataFrame({'price':data_price[:,0]})
data.loc[:, :'price'] = price_df



In [13]:
data.loc[:, :'price'] = price_df
#data.loc[:, :'price']

In [14]:
#data dataframe to np.array vector
data = data.values

In [15]:
#data

In [16]:

# Training and test data
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end + 1
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end), :]

# Build X and y
x_train = data_train[:, 1:]
y_train = data_train[:, 0]
x_test = data_test[:, 1:]
y_test = data_test[:, 0]


In [17]:
# Number of features in training data
n_features = x_train.shape[1]

# Neurons - first layer contains 128 neurons, which is almost the double of the size of the input (66 rows)
# next hidden layers are accounting for half of the size of the previous layer: respectively 64, 32 and 16 neurons
n_neurons_1 = 128
n_neurons_2 = 64
n_neurons_3 = 32
n_neurons_4 = 16

# Session
net = tf.InteractiveSession()

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_features])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_features, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights -  the second dimension of the previous layer is the first dimension in the current layer for weight matrices
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))

# The dimension of the biases = current layer's  weight matrix 2nd dimension
#letter  corresponds to the # of neurons in current layer
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer - widely used rectified linear unit (ReLU) activation is used here
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# MSE computes the average squared error between predictions and targets (known as cost f-n and is a common method in regression analysis) 
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer is adapting the weights and biases during optimization. they are using gradient method (usually gradient descent method)
# to pinpoint the direction of change for weights and biases in order to minimize MSE (cost f-n). Adam optimizier is kind of a default one
opt = tf.train.AdamOptimizer().minimize(mse)

# Init
net.run(tf.global_variables_initializer())



# Fit neural net
batch_size = 8
mse_train = []
mse_test = []

# Run
epochs = 10
for e in range(epochs):

    # Shuffle training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    x_train = x_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    # Minibatch training
    for i in range(0, len(y_train) // batch_size):
        start = i * batch_size
        batch_x = x_train[start:start + batch_size]
        batch_y = y_train[start:start + batch_size]
        # Run optimizer with batch
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        # Show progress
        if np.mod(i, 50) == 0:
            # MSE train and test
            mse_train.append(net.run(mse, feed_dict={X: x_train, Y: y_train}))
            mse_test.append(net.run(mse, feed_dict={X: x_test, Y: y_test}))
            print('MSE Train: ', mse_train[-1])
            print('MSE Test: ', mse_test[-1])
            # Prediction
            pred = net.run(out, feed_dict={X: x_test})
            #line2.set_ydata(pred)
            #plt.title('Epoch ' + str(e) + ', Batch ' + str(i))
            #plt.pause(0.01)

MSE Train:  66.85294
MSE Test:  257.88403
MSE Train:  91.46224
MSE Test:  304.35422
MSE Train:  10.281862
MSE Test:  106.51884
MSE Train:  6.166522
MSE Test:  62.880917
MSE Train:  9.148374
MSE Test:  43.321594
MSE Train:  11.449583
MSE Test:  36.520695
MSE Train:  6.0469947
MSE Test:  65.78923
MSE Train:  8.5991125
MSE Test:  97.53682
MSE Train:  11.963456
MSE Test:  114.39124
MSE Train:  10.759615
MSE Test:  108.98314


In [18]:
pred * 1000

array([[8647.751, 8655.655, 8704.563, 8704.563, 8704.563, 8869.4  ,
        8820.45 , 8869.4  , 8918.309, 8554.241, 8652.122, 8652.122,
        8652.122]], dtype=float32)

In [ ]:
# working on finding the root cause for this error.